In [ ]:
from sentinelsat import *
from collections import OrderedDict
from datetime import datetime,timedelta, date
import pandas as pd
import getpass
import os 
import re
from glob import glob
import subprocess
import numpy as np
from tqdm import tqdm,tqdm_notebook
import zipfile
# from google.cloud import storage 
# import ee
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import requests
import time

In [ ]:
#user authentication (Copernicus account)
username = getpass.getpass("Username:")
pswd = getpass.getpass("Password:")
api = SentinelAPI(username,pswd,'https://scihub.copernicus.eu/dhus')

In [ ]:
def get_products(dates,tiles):
    products = OrderedDict()
    for tile in tiles:
        for date in tqdm(dates,position=0, leave=True):
            date_format = datetime.strptime(date.replace('-',''),'%Y%m%d').date()

            kw_query = {'platformname':'Sentinel-2',
                       'filename':f'*_T{tile}_*',
                       'date':(date_format,date_format+timedelta(days=1))}

            if date_format>pd.to_datetime('2018-12-01').date():
                kw_query['producttype']= 'S2MSI2A'
            else:
                kw_query['producttype']= 'S2MSI1C'
            
            
            pp = api.query(**kw_query)
            products.update(pp)
    return api.to_dataframe(products)

def check_online(products) :
    count = 0
    for product_id in products.index:
        if api.get_product_odata(product_id)['Online']:
            count+=1
    print('Nr of products:',len(products),'online products:',count)
    
def filter_blobs(blobs,band_names):
    band_blobs = []
    for band in band_names:
        for blob in blobs:
            if band in blob.name and blob.name.endswith('.jp2'):
                band_blobs.append(blob)
    return band_blobs
    
# prod_2017 = get_products(dates[0],tiles)
# prod_2018 = get_products(dates[1],tiles)
# prod_2019 = get_products(dates[2],tiles)

# #export as csv
# prod_2017.to_csv(download_dir+'/prod_2017.csv')
# prod_2018.to_csv(download_dir+'/prod_2018.csv')
# prod_2019.to_csv(download_dir+'/prod_2019.csv')

In [ ]:
tiles = ['36JVP','36JVQ']

band_names = ['B02','B03','B04','B05','B06','B07','B08','B8A','B11','B12']

files = glob('./data/ts_animation/*.csv')
dates = list(map(lambda x:pd.read_csv(x,index_col=0).values.reshape(-1),files))
download_dir = getpass.getpass('download dir:')

In [ ]:
s2_index = download_dir+'/index.csv.gz'
google_scenes = pd.read_csv(s2_index, compression='gzip',dtype='unicode')
google_scenes['SENSING_TIME'] = pd.to_datetime(google_scenes['SENSING_TIME']).dt.date.astype(str)

In [ ]:
my_products = google_scenes[google_scenes['SENSING_TIME'].isin(dates[0]) & google_scenes['MGRS_TILE'].isin(tiles)]
my_products = my_products.sort_values('SENSING_TIME')

In [ ]:
client = storage.Client().from_service_account_json(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
bucket = client.get_bucket("gcp-public-data-sentinel-2")

In [ ]:
for base_url in tqdm(my_products['BASE_URL'].values,position=0, leave=True):
    time.sleep(1)
    
    bucket_dest = base_url.replace('gs://gcp-public-data-sentinel-2/','')
    safe_name = bucket_dest.split('/')[-1]
    safe_dir = os.path.join(download_dir,'s2_2017',safe_name)
    
    if not os.path.exists(safe_dir):
        blobs = list(bucket.list_blobs(prefix=bucket_dest))
        blobs = filter_blobs(blobs,band_names)
        for b in blobs:
            bucket_base = 'https://storage.googleapis.com/gcp-public-data-sentinel-2/'
            fend = b.name.split(safe_name)[-1]
            fname = safe_dir+fend
            os.makedirs(os.path.dirname(fname),exist_ok=True)

            with open(fname, 'wb') as f:
                client.download_blob_to_file(b,f)

In [ ]:
#download products
api.download_all(prod_df.index,download_dir+'\\s2_2017')

In [ ]:
#unzip files
# for f in glob(download_dir +'/*.zip'):
#     with zipfile.ZipFile(f, 'r') as zip_ref:
#         zip_ref.extractall(download_dir)